## Data Cleaning
This notebook explains the steps that are run in the file **load.py**.

In [24]:
import pandas as pd
import numpy as np
import scipy as sp
import scipy.stats as stats

As mentioned in the README, I use Stanford's **Database on Ideology, Money in Politics, and Elections (DIME)**. This is a comprehensive collection that combines campaign finance data, election results, and other relevant information. Many of the columns our not needed for the purpose of my exploration. If interested in further exploration, see the DIME Codebook for details on the full dataset. 

In [113]:
df1 = pd.read_csv('../../Downloads/dime_recipients_all_1979_2014.csv', low_memory=False)

In [114]:
df2 = pd.read_csv('../../Downloads/dime_recipients_1979_2014.csv', low_memory=False)

In [117]:
### Load the original dataset
results_raw = pd.read_csv('data/dime.csv', low_memory=False)

In [26]:
### View all column names
results_raw.columns

Index(['election', 'cycle', 'fecyear', 'Cand.ID', 'FEC.ID', 'NID', 'ICPSR',
       'ICPSR2', 'bonica.rid', 'bonica.cid', 'name', 'lname', 'ffname',
       'fname', 'mname', 'nname', 'title', 'suffix', 'party', 'state', 'seat',
       'district', 'Incum.Chall', 'recipient.cfscore', 'contributor.cfscore',
       'recipient.cfscore.dyn', 'dwnom1', 'dwnom2', 'ps.dwnom1', 'ps.dwnom2',
       'dwdime', 'irt.cfscore', 'num.givers', 'num.givers.total',
       'n.data.points.personal.donations',
       'n.data.points.personal.donations.unq', 'cand.gender',
       'total.disbursements', 'total.pc.contribs', 'contribs.from.candidate',
       'unitemized', 'non.party.ind.exp.for', 'non.party.ind.exp.against',
       'ind.exp.for', 'ind.exp.against', 'comm.cost.for', 'comm.cost.against',
       'party.coord.exp', 'party.ind.exp.against', 'total.receipts',
       'total.indiv.contrib', 'total.pac.contribs', 'ran.primary',
       'ran.general', 'p.elec.stat', 's.elec.stat', 'r.elec.stat',
       'gen

### Dataset info
Each row represents a candidate in a specific election. **Cand.ID** provides a candidate ID, which is consistent from one election to the next. **ICPSR**, however, is unique to each candidate and each election. This will be important to have later.

* ICPSR stands for Inter-university Consortium for Political and Social Research
* This column provides the **adjusted ICPSR legislator ID**. 
* Details from DIME Codebook: "Candidates that have never served in Congress are assigned IDs based off of their FEC IDs. The four-digit election cycle is appended to the end of each ID to distinguish separate entries from the same candidate. Candidates that are active in multiple election cycles will have multiple IDs"


### Narrow the fields
I chose 26 columns that are most pertinent for my study. 

In [27]:
col_list = ['election', 'cycle', 'Cand.ID', 'ICPSR', 'name', 'lname', 'fname','party', 
            'state', 'seat', 'district', 'Incum.Chall', 'num.givers', 'num.givers.total', 'cand.gender',
            'total.disbursements', 'total.pc.contribs', 'contribs.from.candidate', 'total.receipts',
            'total.indiv.contrib', 'total.pac.contribs', 'ran.general','gen.elec.stat', 
            'gen.elect.pct', 'winner', 'district.partisanship']
len(col_list)

26

In [79]:
### Narrow to selected columns
results = results_raw.loc[:, col_list]
### Limit dataset to candidates in the General Election
results = results[results['ran.general']==1]
### Exclude rows that do not have values for the percentage of vote received in general election
results = results[~results['gen.elect.pct'].isna()]
### Limit dataset to House elections
results = results[results.seat == 'federal:house']

In [80]:
results.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19090 entries, 89 to 47538
Data columns (total 26 columns):
election                   19090 non-null object
cycle                      19090 non-null int64
Cand.ID                    19090 non-null object
ICPSR                      19090 non-null object
name                       19090 non-null object
lname                      19089 non-null object
fname                      19081 non-null object
party                      19090 non-null object
state                      19090 non-null object
seat                       19090 non-null object
district                   19090 non-null object
Incum.Chall                19090 non-null object
num.givers                 18832 non-null float64
num.givers.total           19090 non-null int64
cand.gender                19081 non-null object
total.disbursements        19090 non-null float64
total.pc.contribs          19090 non-null float64
contribs.from.candidate    19090 non-null float64
total.

In [81]:
results.iloc[:3,:15]

,election,cycle,Cand.ID,ICPSR,name,lname,fname,party,state,seat,district,Incum.Chall,num.givers,num.givers.total,cand.gender
89,fd1980,1980,H6PA07022,142171980,"edgar, robert w.",edgar,robert,100,PA,federal:house,PA07,I,61.0,670,M
92,fd1980,1980,H6PA24043,144361980,"marks, marc lincoln",marks,marc,200,PA,federal:house,PA24,I,45.0,46,M
97,fd1980,1980,H6OK06036,142191980,"english, glenn lee jr",english,glenn,100,OK,federal:house,OK06,I,33.0,229,M


In [82]:
results.iloc[:3,15:].head()

,total.disbursements,total.pc.contribs,contribs.from.candidate,total.receipts,total.indiv.contrib,total.pac.contribs,ran.general,gen.elec.stat,gen.elect.pct,winner,district.partisanship
89,15794.0,0.0,0.0,15795.0,11004.0,4791.0,1.0,L,0,L,0.599
92,10740.0,0.0,0.0,10741.0,10741.0,0.0,1.0,L,0,L,-0.024
97,3669.0,0.0,0.0,3702.0,3202.0,500.0,1.0,L,0,L,0.931


Now that we can view the basic structure of the data we have, it is worth considering what structure we want to achieve. 
### Remember the research question: 
In congressional rematch elections, is there a correlation between changes in campaign expenditures and changes in vote percentage? 

#### Defining 'rematch'
For clarity, we'll think of rematches as two elections that have the same two leading candidates. When we look at the change in campaign spending and votes received, we will evaluate the races chronologically. 

#### Planning the database
We currently have one row per candidate per election. We want one row per rematch, which includes candidate A, candidate B, first race, and second race. 

### Engineering our Database :
#### As a preliminary step towards finding the top two candidates in any given race, we need to create a column that specifies both year & district.

In [83]:
results['race_ID'] = results['cycle'].astype(str) + "-" + results['district'].astype(str)

#### Now we'll use the race_ID column to find the elections for which we still have info on at least 2 candidates. 

In [84]:
### When trying to change data type of vote percentage column, I found string values
results = results[results['gen.elect.pct'] != '?? ']
### Check number of candidates per race (after removing rows that lack numeric values for vote percentage)
num_cand = results.race_ID.value_counts()
### Create a dataframe to hold the race_IDs for which we still have multiple candidates
num_cand_df = pd.DataFrame()
num_cand_df = num_cand[num_cand>1]
### Use the new dataframe to select all of the candidate rows for the selected race_IDs
results = results[results['race_ID'].isin(num_cand_df.reset_index()['index'])]

Convert the dtype for vote percentage column to **float**. This is more efficient, and will later necessary be for calculating the change in vote percentage from one election to the next? 

In [85]:
results.loc[:,'gen.elect.pct'] = results['gen.elect.pct'].astype(float).copy()

In [86]:
results.head()

,election,cycle,Cand.ID,ICPSR,name,lname,fname,party,state,seat,...,contribs.from.candidate,total.receipts,total.indiv.contrib,total.pac.contribs,ran.general,gen.elec.stat,gen.elect.pct,winner,district.partisanship,race_ID
89,fd1980,1980,H6PA07022,142171980,"edgar, robert w.",edgar,robert,100,PA,federal:house,...,0.0,15795.0,11004.0,4791.0,1.0,L,0.0,L,0.599,1980-PA07
92,fd1980,1980,H6PA24043,144361980,"marks, marc lincoln",marks,marc,200,PA,federal:house,...,0.0,10741.0,10741.0,0.0,1.0,L,0.0,L,-0.024,1980-PA24
104,fd1980,1980,H6OK05038,144151980,"edwards, marvin h ""mickey""",edwards,marvin,200,OK,federal:house,...,0.0,88046.0,86546.0,1500.0,1.0,L,0.0,L,1.261,1980-OK05
160,fd1980,1980,H6CA26027,105531980,"rousselot, john h",rousselot,john,200,CA,federal:house,...,0.0,237769.0,180779.0,56990.0,1.0,L,0.0,L,-0.126,1980-CA26
197,fd1980,1980,H8WV03014,148651980,"staton, david michael ""mick""",staton,david,200,WV,federal:house,...,0.0,70755.0,25572.0,32321.0,1.0,W,53.0,W,-0.342,1980-WV03


In [87]:
results.loc[13800, 'gen.elect.pct'] = 48.01
results.loc[23340, 'gen.elect.pct'] = 45.01

We can use the **nlargest** method to find the two leading candidates in each race. By default, this will order the top two, starting with largest. This will help later when we split the candidates into groups A & B. 

In two instances, we will find the top two candidates have the same vote percentages, as the values are rounded. To ensure all of the winning candidates are placed in group A, I added 0.01 to the vote percentage of the winning candidate in those two races. 

In [88]:
### Find top 2 candidates per race 
top2_cand = results.groupby('race_ID')['gen.elect.pct'].nlargest(2)
print(len(top2_cand))

11608


#### Take a look at the resulting pairs of candidates. 
While we don't see names, we have indices and the percentages of votes received. Just viewing the first 5 pairs, we can identify a problem with our data. 

In [89]:
top2_cand.head(10)

race_ID        
1980-AK01  906     74.0
           1223    26.0
1980-AL02  1222    61.0
           905     37.0
1980-AL06  327     51.0
           904     47.0
1980-AZ01  509     73.0
           902      0.0
1980-AZ04  508     63.0
           321     37.0
Name: gen.elect.pct, dtype: float64

### Troubleshooting: 
Note the results for the 1980 election for Arizona's first congressional district (**1980-AZ01**). While the vote percentages are rounded and 0.0% is a viable result for some candidates, it is unlikely that the second-place candidate received so few votes, especially when the winning candidate only received 73% of the vote. 

My best guess is that the data is missing entries for one or more candidates in this race, which resulted in the 'Top 2' candidate rankings to include a candidate who actually placed 3rd, 4th, or worse. Alternatively, it is possible that the vote percentage for some candidates was incorrectly entered as 0%. 

This is a natural consequence of working with an incomplete and/or imperfect dataset, but by doing some basic research, we can minimize the impact these issues have on our results.

#### Now that we suspect a problem, is worth checking if this is an anomaly or a frequent occurence. 

In [99]:
### How often does one of the 'top candidates' have a vote percentage of 0?
print("How often does one of the 'top candidates' have a vote percentage of 0? ", 
      len(top2_cand[top2_cand.values == 0]), 'times')
top2_cand[top2_cand.values == 0].head(10)

How often does one of the 'top candidates' have a vote percentage of 0?  487 times


race_ID       
1980-AZ01  902    0.0
1980-CA26  160    0.0
1980-CA38  868    0.0
1980-CA41  500    0.0
1980-FL14  298    0.0
1980-ID01  810    0.0
1980-IL01  294    0.0
1980-IL05  802    0.0
1980-IL13  795    0.0
1980-KY05  755    0.0
Name: gen.elect.pct, dtype: float64

#### Since this occurs a few too many times for comfort. Let's learn more about a few instances.
While it is too time consuming to look into every instance, we can do some quick research to see if we can determine whether we have missing candidates or incorrect vote percentages. 

#### I chose 3 of the first 10 examples: the 1980 elections for the congressional districts AZ 1st, CA  26th, and FL 14th. 
We want to see all of the candidate entries included in the original data for these races. 

While I have quadruple-checked the code up to this point, it is a good idea to look in **results_raw**, to view all of the candidate entries included in the original data. This would help us catch mistakes in case I inadvertantly filtered out some of the candidates we're interested in. 

In [100]:
print('Here are the candidates included in the original data for the three races mentioned above:')
display(results_raw[(results_raw.district == 'AZ01')&(results_raw.cycle == 1980)][['cycle','name','district','gen.elect.pct']])
display(results_raw[(results_raw.district == 'CA26')&(results_raw.cycle == 1980)][['cycle','name','district','gen.elect.pct']])
display(results_raw[(results_raw.district == 'FL14')&(results_raw.cycle == 1980)][['cycle','name','district','gen.elect.pct']])

Here are the candidates included in the original data for the three races mentioned above:


,cycle,name,district,gen.elect.pct
509,1980,"rhodes, john",AZ01,73
902,1980,"amacher, ryan custer",AZ01,0


,cycle,name,district,gen.elect.pct
160,1980,"rousselot, john h",CA26,0
877,1980,"lisoni, joseph louis",CA26,24


,cycle,name,district,gen.elect.pct
298,1980,"miller, phyllis",FL14,0
1189,1980,"pepper, claude",FL14,75


#### A quick search allows us to compare official election results to the information in the dataset. 
You can download official [election results as a PDF](https://history.house.gov/Institution/Election-Statistics/1980election/)  from the House of Representatives Archives. 

| District |     Candidate Name         |      Party        |  %   |
|:--------:|:--------------------------:|:-----------------:|:----:|
| AZ01     | John Jacob Rhodes          | Republican        | 73.3 |
| AZ01     | Steve Jancek               | Democratic        | 21.4 |
| AZ01     | Irene Leitch               | Libertarian       | 4.2  |
| AZ01     | Rob Roper                  | Socialist Workers | 1.1  |
|    -     |             -              |         -         |  -   |
| CA26     | John H. Rousselot          | Republican        | 70.9 |
| CA26     | Joseph Louis Lisoni        | Democratic        | 24.4 |
| CA26     | William J. "B. J." Wagener | Libertarian       | 4.7  |
|    -     |             -              |         -         |  -   |
| FL14     | Claude Pepper              | Democratic        | 74.9 |
| FL14     | Evelio S. Estrella         | Republican        | 25.1 |

With only three sample races, we find 5 candidates that were not included in the dataset (Jancek, Leitch, Roper, Wagener, and Estrella). Only Jancek and Estrella, however, would have an impact on our immediate question, as the others would have been excluded from the "Top 2" search regardless. We also found one instance (Rousselot) of an incorrect value entered for vote percentage; the 0% recorded in the dataset should have been 70.9%.

It is not an efficient use of time to double check all of the election results manually, so **my short-term solution will be to remove the races in which one of the Top 2 candidates received 0% of the vote**. A longer-term solution will be to utilize alternative sources of data, either in place or in addition to the DIME data.

### Time to move on from troubleshooting, and continue building our dataset...

#### Reminder: We are looking for elections that have the same two leading candidates.
The previous step was to identify the Top 2 candidates for every race. Our next step is to find groups (usually pairs) of races that have the same top candidates. 

In [102]:
### 'top2_cand' is a Pandas Series double indexed by 'race_ID' and row index (in the dataframe 'results')
for x in range(10):
    print(top2_cand.index[x])

('1980-AK01', 906)
('1980-AK01', 1223)
('1980-AL02', 1222)
('1980-AL02', 905)
('1980-AL06', 327)
('1980-AL06', 904)
('1980-AZ01', 509)
('1980-AZ01', 902)
('1980-AZ04', 508)
('1980-AZ04', 321)


In [106]:
### Use the row index to get the Candidate ID for every entry in 'top2_cand'
### Make dictionary with keys as 'race_ID' and values as 'Cand.ID'
cand_pair_dct = {}
for pair in top2_cand.index:
    race_ID, idx = pair[0], pair[1]
    cand_id = results.loc[idx, 'Cand.ID']
    if race_ID not in cand_pair_dct:
        cand_pair_dct[race_ID] = []
    cand_pair_dct[race_ID].append(cand_id)

In [107]:
### Repeat the above process using the column 'ICPSR'
cand_pair_dct2 = {}
for pair in top2_cand.index:
    cand_id = results.loc[pair[1], 'ICPSR']
    if pair[0] not in cand_pair_dct2:
        cand_pair_dct2[pair[0]] = []
    cand_pair_dct2[pair[0]].append(cand_id)

We made two dictionaries, 'cand_pair_dct' and 'cand_pair_dct2'. They are similar, but can't be mixed up!
* cand_pair_dct uses 'Cand.ID'. We need an ID for each candidate that is the same for every race. We will make a **'top2' column** that we can use to identify rematches. 
* cand_pair_dct2 uses 'ICPSR'. Because each candidate may have a row for every election they have participated in, we will likely find that some candidates from rematch races will also appear in rows for non-rematch races. 'ICPSR' is a candidate ID that is unique to each election, so we can identify the correct rows for a given candidate.  

In [108]:
### Make the top2 column 
results.loc[:,'top2'] = results['race_ID'].copy().apply(lambda x: cand_pair_dct[x])

#### Make a reverse dictionary from 'cand_paid_dct':
Use the new column of Top 2 candidate IDs as keys. Each value is the set of race_IDs that share the same two leading candidates. 

In [109]:
rev_dict = {}
for key, value in cand_pair_dct.items():
    rev_dict.setdefault(str(value), set()).add(key)
### Make a list of lists. Each list has race_IDs that share the same Top 2 candidates. 
rematch_races = [sorted(values) for key, values in rev_dict.items() if len(values) > 1]

Now we have a list of rematches. It includes all instances when the same Top 2 candidates appear in more than one race.

In [111]:
rematch_races[0:10]

[['1980-CA09', '1982-CA09'],
 ['1980-CA11', '1982-CA11'],
 ['1980-CA36', '1982-CA36', '1984-CA36', '1988-CA36'],
 ['1980-CA37', '1992-CA40', '1994-CA40'],
 ['1980-CT02', '1982-CT02'],
 ['1980-CT04', '1982-CT04'],
 ['1980-IA03', '1982-IA03'],
 ['1980-IL17', '1982-IL04'],
 ['1980-IN10', '1992-IN02'],
 ['1980-KY02', '1982-KY02']]

In [112]:
rematch_cand = [list(values) for values in cand_pair_dct2.values()]
rematch_cand[0:10]

[['H0AK010121980', '140661980'],
 ['107171980', 'H0AL020121980'],
 ['148621980', 'H0AL060131980'],
 ['78451980', 'H0AZ010101980'],
 ['144491980', 'H8AZ040171980'],
 ['148051980', '49571980'],
 ['105791980', 'H8CA020161980'],
 ['140761980', 'H0CA050161980'],
 ['105771980', 'H6CA060601980'],
 ['142561980', 'H0CA070201980']]

In [21]:
rematch_cand_A = [each[0] for each in rematch_cand if len(each)==2]
rematch_cand_B = [each[1] for each in rematch_cand if len(each)==2]

In [22]:
rematch_race_id = []
for y in rematch_sets:
    for x in y:
        rematch_race_id.append(x)

rematch_cand_id = []
for y in rematch_cand:
    for x in y:
        rematch_cand_id.append(x)    

rematch_cand_id = list(set(rematch_cand_id))

In [23]:
print(len(rematch_race_id))
rematch_race_id

1180


['1980-CA09',
 '1982-CA09',
 '1980-CA11',
 '1982-CA11',
 '1980-CA36',
 '1982-CA36',
 '1984-CA36',
 '1988-CA36',
 '1980-CA37',
 '1992-CA40',
 '1994-CA40',
 '1980-CT02',
 '1982-CT02',
 '1980-CT04',
 '1982-CT04',
 '1980-IA03',
 '1982-IA03',
 '1980-IL17',
 '1982-IL04',
 '1980-IN10',
 '1992-IN02',
 '1980-KY02',
 '1982-KY02',
 '1980-MA02',
 '1982-MA02',
 '1984-MA02',
 '1980-MA06',
 '1982-MA06',
 '1980-MD02',
 '1982-MD02',
 '1980-MO01',
 '1982-MO01',
 '1980-NC05',
 '1982-NC05',
 '1980-NE02',
 '1982-NE02',
 '1980-NJ03',
 '1982-NJ03',
 '1980-NM01',
 '1984-NM01',
 '1980-OH05',
 '1982-OH05',
 '1984-OH05',
 '1980-OH14',
 '1982-OH14',
 '1980-OK03',
 '1982-OK03',
 '1980-OK04',
 '1982-OK04',
 '1980-TX13',
 '1982-TX13',
 '1980-TX21',
 '1984-TX21',
 '1980-UT01',
 '1986-UT01',
 '1988-UT01',
 '1980-VA08',
 '1982-VA08',
 '1980-WA05',
 '1982-WA05',
 '1992-WA05',
 '1982-CA04',
 '1984-CA04',
 '1982-CA10',
 '1984-CA10',
 '1982-CA24',
 '1984-CA24',
 '1982-CT03',
 '1984-CT03',
 '1982-IL16',
 '1984-IL16',
 '1982

In [24]:
print(len(rematch_cand_id))
rematch_cand_id

11717


['293182004',
 '209082008',
 '151111984',
 '295441998',
 '140721998',
 '150631986',
 '293932010',
 'H0NM010981990',
 'H6OH060531988',
 'H6FL040631996',
 '154331986',
 'H0GA030332000',
 '130401980',
 '293112000',
 'H8AZ080182008',
 '150361994',
 '207262006',
 '293182000',
 'H0NY200531990',
 '299171998',
 'H6NY291011988',
 'H8UT010591998',
 'H8NY220521998',
 '297462002',
 'H0CA030782010',
 'H0CA301052010',
 'H8NJ010511988',
 '146311988',
 'H0TX190432010',
 '144352012',
 'H0TX240191980',
 '209272010',
 'H0SC061952010',
 'H2FL061092014',
 '150332000',
 '295021998',
 'H0SC042572012',
 '297072006',
 '150502002',
 '156191994',
 '146081986',
 'H6FL021411998',
 '154381998',
 '150321992',
 '293851994',
 'H2MA100391992',
 '207272006',
 'H8MN050981998',
 '151291988',
 '154441988',
 '291001998',
 '293462008',
 '148692002',
 '148061984',
 'H4CA080792004',
 '150262000',
 'H0OK040831982',
 '297632004',
 '142901980',
 '291431992',
 '291371996',
 '293451996',
 '148101980',
 'H4PA080582004',
 '150641998'

In [27]:
results = results[results['ICPSR'].isin(rematch_cand_id)]

In [28]:
match_one = [x[0] for x in rematch_sets]
match_two = [x[1] for x in rematch_sets]

In [44]:
### Drop this
#'2004-AS01'